# Logistic Regression to Classify Terrain by IMU and Odometry Data from TurtleBot3
### By Jacob Laframboise, Jack Demeter
Logistic regression works great when the data is randomly split into train and test (high 90 accuracy), but it struggles when the data is split into train/test based on which trial number is was collected in. This further supports the hypothesis that data from each run is more similar to itself than data from a given terrain. 



In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.feature_selection import SelectKBest, chi2

import plotly as ply
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib notebook

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 80)

In [2]:
## Data Pre-Processing
#### We load in individual csv files collected from Rosbag on TurtleBot3 with ROS.

In [3]:
dataFolder = r"C:\Users\jaker\Documents\Experiment3Data-2019-11-21"

dataFiles = [
    r"gMitTile_s15_t8.csv",
    r"gMitTile_s15_t9.csv",
    r"gMitTile_s15_t10.csv",
    r"gTurf_s15_t3.csv",
    r"gTurf_s15_t4.csv",
    r"gTurf_s15_t5.csv",
    r"gTurf_s15_t6.csv",
    r"gTurf_s15_t7.csv",
    r"gTurf_s15_t8.csv",
    r"gTurf_s15_t9.csv",
    r"gTurf_s15_t10.csv",
    r"gArcTile_s15_t3.csv",
    r"gArcTile_s15_t4.csv",
    r"gArcTile_s15_t5.csv",
    r"gArcTile_s15_t6.csv",
    r"gArcTile_s15_t7.csv",
    r"gArcTile_s15_t8.csv",
    r"gArcTile_s15_t9.csv",
    r"gArcTile_s15_t10.csv",
    r"gCarp_s15_t3.csv",
    r"gCarp_s15_t4.csv",
    r"gCarp_s15_t5.csv",
    r"gCarp_s15_t6.csv",
    r"gCarp_s15_t7.csv",
    r"gCarp_s15_t8.csv",
    r"gCarp_s15_t9.csv",
    r"gCarp_s15_t10.csv",
    r"gMitTile_s15_t3.csv",
    r"gMitTile_s15_t4.csv",
    r"gMitTile_s15_t5.csv",
    r"gMitTile_s15_t6.csv",
    r"gMitTile_s15_t7.csv"
]

savePath = "Data-32Series-Delta30-Squared.csv"

In [4]:
""" 
For each data file we:
adjust the index, 
interpolate NaN values,
drop remaining NaN values, 
drop some empty columns.

We then augment the feature space with delta columns, 
and with polynomial columns,
and label the columns with terrain, speed, and trial number
"""
for i in range(len(dataFiles)):
    terrain = dataFiles[i].split('_')[0][1:]
    speed = dataFiles[i].split('_s')[1][:2]
    trial = dataFiles[i].split('_t')[1][0]

    df = pd.read_csv(os.path.join(dataFolder, dataFiles[i]))
    df = df.rename(columns={'Unnamed: 0': 'Seq'})
    df = df.set_index('Seq')

    #print(df.isnull().sum().sum())
    df = df.interpolate(method='polynomial', order=1)
    #print(df.isnull().sum().sum())

    df = df.dropna()
    #print(df.isnull().sum().sum())
    df = df.reset_index().drop(columns=['Seq'])

    df = df.drop(columns=['OdomPosZ', 'OdomOrientX', 'OdomOrientY', 'OdomLinY', 'OdomLinZ', 'OdomAngX', 'OdomAngY'])

    order=3
    
    dList = range(1, 302, 10)
    # dList = [2,4,8,16,32,64]
    for col in df.columns.tolist():
        if col!='Sensor':
            for d in dList:
                df[col+'Delta{}'.format(d)] = df[col].diff(d)
                if order>1:
                    for p in range(2, order+1):
                        df[col+'Delta{}Exp{}'.format(d, p)] = df[col+'Delta{}'.format(d)]**p
                        #print('Added ' + col +'Delta{}'.format(d))
        else:
            pass

    df = df.iloc[max(dList):].reset_index().drop(columns=['index'])
    df = df.drop(columns=['Sensor', 'Time'])
    df['Speed']=int(speed)
    df['Terrain']=terrain
    df['Trial']=int(trial)

    if i==0:
        mainDf = df.copy(deep=True)
    else:
        mainDf = pd.concat([mainDf, df], axis=0, sort=False)
    print("Added {} of size {} to mainDf. ".format(dataFiles[i], df.shape))
    print("Data series completed: {}/{}".format(i+1,len(dataFiles)))
    print("MainDf is now size {}".format(mainDf.shape))
    print('\n')


Added gMitTile_s15_t8.csv of size (3530, 1600) to mainDf. 
Data series completed: 1/32
MainDf is now size (3530, 1600)


Added gMitTile_s15_t9.csv of size (4030, 1600) to mainDf. 
Data series completed: 2/32
MainDf is now size (7560, 1600)


Added gMitTile_s15_t10.csv of size (4552, 1600) to mainDf. 
Data series completed: 3/32
MainDf is now size (12112, 1600)


Added gTurf_s15_t3.csv of size (5800, 1600) to mainDf. 
Data series completed: 4/32
MainDf is now size (17912, 1600)


Added gTurf_s15_t4.csv of size (5539, 1600) to mainDf. 
Data series completed: 5/32
MainDf is now size (23451, 1600)


Added gTurf_s15_t5.csv of size (3051, 1600) to mainDf. 
Data series completed: 6/32
MainDf is now size (26502, 1600)


Added gTurf_s15_t6.csv of size (4952, 1600) to mainDf. 
Data series completed: 7/32
MainDf is now size (31454, 1600)


Added gTurf_s15_t7.csv of size (4950, 1600) to mainDf. 
Data series completed: 8/32
MainDf is now size (36404, 1600)


Added gTurf_s15_t8.csv of size (6367, 16

In [12]:
# avoid reloading dataset when working
df = mainDf.copy(deep=True)

In [13]:
df.head(10)

,OdomPosX,OdomPosY,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,TimeDelta1,TimeDelta1Exp2,TimeDelta1Exp3,TimeDelta11,TimeDelta11Exp2,TimeDelta11Exp3,TimeDelta21,TimeDelta21Exp2,TimeDelta21Exp3,TimeDelta31,TimeDelta31Exp2,TimeDelta31Exp3,TimeDelta41,TimeDelta41Exp2,TimeDelta41Exp3,TimeDelta51,TimeDelta51Exp2,TimeDelta51Exp3,TimeDelta61,TimeDelta61Exp2,TimeDelta61Exp3,TimeDelta71,TimeDelta71Exp2,TimeDelta71Exp3,TimeDelta81,TimeDelta81Exp2,TimeDelta81Exp3,TimeDelta91,TimeDelta91Exp2,TimeDelta91Exp3,TimeDelta101,TimeDelta101Exp2,TimeDelta101Exp3,TimeDelta111,TimeDelta111Exp2,TimeDelta111Exp3,TimeDelta121,TimeDelta121Exp2,TimeDelta121Exp3,TimeDelta131,TimeDelta131Exp2,TimeDelta131Exp3,TimeDelta141,TimeDelta141Exp2,...,ImuAccelZDelta121,ImuAccelZDelta121Exp2,ImuAccelZDelta121Exp3,ImuAccelZDelta131,ImuAccelZDelta131Exp2,ImuAccelZDelta131Exp3,ImuAccelZDelta141,ImuAccelZDelta141Exp2,ImuAccelZDelta141Exp3,ImuAccelZDelta151,ImuAccelZDelta151Exp2,ImuAccelZDelta151Exp3,ImuAccelZDelta161,ImuAccelZDelta161Exp2,ImuAccelZDelta161Exp3,ImuAccelZDelta171,ImuAccelZDelta171Exp2,ImuAccelZDelta171Exp3,ImuAccelZDelta181,ImuAccelZDelta181Exp2,ImuAccelZDelta181Exp3,ImuAccelZDelta191,ImuAccelZDelta191Exp2,ImuAccelZDelta191Exp3,ImuAccelZDelta201,ImuAccelZDelta201Exp2,ImuAccelZDelta201Exp3,ImuAccelZDelta211,ImuAccelZDelta211Exp2,ImuAccelZDelta211Exp3,ImuAccelZDelta221,ImuAccelZDelta221Exp2,ImuAccelZDelta221Exp3,ImuAccelZDelta231,ImuAccelZDelta231Exp2,ImuAccelZDelta231Exp3,ImuAccelZDelta241,ImuAccelZDelta241Exp2,ImuAccelZDelta241Exp3,ImuAccelZDelta251,ImuAccelZDelta251Exp2,ImuAccelZDelta251Exp3,ImuAccelZDelta261,ImuAccelZDelta261Exp2,ImuAccelZDelta261Exp3,ImuAccelZDelta271,ImuAccelZDelta271Exp2,ImuAccelZDelta271Exp3,ImuAccelZDelta281,ImuAccelZDelta281Exp2,ImuAccelZDelta281Exp3,ImuAccelZDelta291,ImuAccelZDelta291Exp2,ImuAccelZDelta291Exp3,ImuAccelZDelta301,ImuAccelZDelta301Exp2,ImuAccelZDelta301Exp3,Speed,Terrain,Trial
0,-3.813970,18.415197,0.095387,0.995440,0.148886,-0.006150,-0.004592,-0.004974,-0.104227,-0.994526,0.000000,-0.027669,-0.013835,-0.290297,-0.049081,9.154829,0.005092,0.000026,1.320215e-07,0.065911,0.004344,0.000286,0.130681,0.017078,0.002232,0.187764,0.035255,0.006620,0.251098,0.063050,0.015832,0.312472,0.097639,0.030509,0.368369,0.135695,0.049986,0.425546,0.181089,0.077062,0.493427,0.243470,0.120135,0.557480,0.310784,0.173256,0.636363,0.404958,0.257700,0.716054,0.512733,0.367144,0.771192,0.594737,0.458657,0.834672,0.696677,0.581497,0.892386,0.796352,...,-2.354098,5.541779,-13.045892,-0.283712,0.080493,-0.022837,-3.133411,9.818267,-30.764670,0.202311,0.040930,0.008280,-1.962646,3.851981,-7.560077,-1.888626,3.566907,-6.736553,-1.224035,1.498262,-1.833926,-1.383249,1.913379,-2.646680,-1.312022,1.721402,-2.258518,-2.908955,8.462017,-24.615623,-0.547673,0.299946,-0.164272,-0.708085,0.501384,-0.355023,-0.365115,0.133309,-0.048673,-2.274491,5.173311,-11.766650,0.575208,0.330864,0.190315,1.256956,1.579939,1.985914e+00,-1.317110,1.734778,-2.284892,-1.206378,1.455348,-1.755700,-1.399111,1.957512,-2.738778,15,MitTile,8
1,-3.813349,18.415318,0.095366,0.995442,0.148886,-0.007478,-0.003933,-0.004679,-0.104174,-0.994536,0.008514,-0.064916,-0.013835,-0.099958,-0.174777,10.842142,0.005561,0.000031,1.719599e-07,0.065683,0.004314,0.000283,0.130839,0.017119,0.002240,0.188547,0.035550,0.006703,0.251223,0.063113,0.015855,0.307715,0.094688,0.029137,0.368386,0.135708,0.049993,0.425174,0.180773,0.076860,0.493495,0.243537,0.120184,0.557880,0.311230,0.173629,0.634416,0.402483,0.255342,0.715011,0.511240,0.365542,0.770069,0.593006,0.456656,0.832622,0.693260,0.577223,0.892173,0.795973,...,-0.326809,0.106804,-0.034905,1.314716,1.728478,2.272457,0.354342,0.125558,0.044490,1.889624,3.570678,6.747237,1.449689,2.101598,3.046663,-0.358532,0.128545,-0.046088,0.463278,0.214626,0.099432,-0.134075,0.017976,-0.002410,0.762553,0.581487,0.443415,-1.505953,2.267894,-3.415341,1.201290

In [14]:
# check for none values and correct size
print(df.isnull().sum().sum()) # check for NaN
df.shape

0


(145353, 1600)

### Feature engineering: elimination

In [15]:
# eliminate columns derived from time, and from angular velocity
columnsToDrop = [x for x in df.columns.tolist() if 'Time' in x or ('Delta' not in x and 'Speed' not in x and 'Terrain' not in x and 'Trial' not in x) ]
df = df.drop(columns=columnsToDrop)

In [16]:
df.head(8)

,OdomPosXDelta1,OdomPosXDelta1Exp2,OdomPosXDelta1Exp3,OdomPosXDelta11,OdomPosXDelta11Exp2,OdomPosXDelta11Exp3,OdomPosXDelta21,OdomPosXDelta21Exp2,OdomPosXDelta21Exp3,OdomPosXDelta31,OdomPosXDelta31Exp2,OdomPosXDelta31Exp3,OdomPosXDelta41,OdomPosXDelta41Exp2,OdomPosXDelta41Exp3,OdomPosXDelta51,OdomPosXDelta51Exp2,OdomPosXDelta51Exp3,OdomPosXDelta61,OdomPosXDelta61Exp2,OdomPosXDelta61Exp3,OdomPosXDelta71,OdomPosXDelta71Exp2,OdomPosXDelta71Exp3,OdomPosXDelta81,OdomPosXDelta81Exp2,OdomPosXDelta81Exp3,OdomPosXDelta91,OdomPosXDelta91Exp2,OdomPosXDelta91Exp3,OdomPosXDelta101,OdomPosXDelta101Exp2,OdomPosXDelta101Exp3,OdomPosXDelta111,OdomPosXDelta111Exp2,OdomPosXDelta111Exp3,OdomPosXDelta121,OdomPosXDelta121Exp2,OdomPosXDelta121Exp3,OdomPosXDelta131,OdomPosXDelta131Exp2,OdomPosXDelta131Exp3,OdomPosXDelta141,OdomPosXDelta141Exp2,OdomPosXDelta141Exp3,OdomPosXDelta151,OdomPosXDelta151Exp2,OdomPosXDelta151Exp3,OdomPosXDelta161,OdomPosXDelta161Exp2,OdomPosXDelta161Exp3,OdomPosXDelta171,OdomPosXDelta171Exp2,OdomPosXDelta171Exp3,OdomPosXDelta181,OdomPosXDelta181Exp2,OdomPosXDelta181Exp3,OdomPosXDelta191,OdomPosXDelta191Exp2,OdomPosXDelta191Exp3,...,ImuAccelZDelta121,ImuAccelZDelta121Exp2,ImuAccelZDelta121Exp3,ImuAccelZDelta131,ImuAccelZDelta131Exp2,ImuAccelZDelta131Exp3,ImuAccelZDelta141,ImuAccelZDelta141Exp2,ImuAccelZDelta141Exp3,ImuAccelZDelta151,ImuAccelZDelta151Exp2,ImuAccelZDelta151Exp3,ImuAccelZDelta161,ImuAccelZDelta161Exp2,ImuAccelZDelta161Exp3,ImuAccelZDelta171,ImuAccelZDelta171Exp2,ImuAccelZDelta171Exp3,ImuAccelZDelta181,ImuAccelZDelta181Exp2,ImuAccelZDelta181Exp3,ImuAccelZDelta191,ImuAccelZDelta191Exp2,ImuAccelZDelta191Exp3,ImuAccelZDelta201,ImuAccelZDelta201Exp2,ImuAccelZDelta201Exp3,ImuAccelZDelta211,ImuAccelZDelta211Exp2,ImuAccelZDelta211Exp3,ImuAccelZDelta221,ImuAccelZDelta221Exp2,ImuAccelZDelta221Exp3,ImuAccelZDelta231,ImuAccelZDelta231Exp2,ImuAccelZDelta231Exp3,ImuAccelZDelta241,ImuAccelZDelta241Exp2,ImuAccelZDelta241Exp3,ImuAccelZDelta251,ImuAccelZDelta251Exp2,ImuAccelZDelta251Exp3,ImuAccelZDelta261,ImuAccelZDelta261Exp2,ImuAccelZDelta261Exp3,ImuAccelZDelta271,ImuAccelZDelta271Exp2,ImuAccelZDelta271Exp3,ImuAccelZDelta281,ImuAccelZDelta281Exp2,ImuAccelZDelta281Exp3,ImuAccelZDelta291,ImuAccelZDelta291Exp2,ImuAccelZDelta291Exp3,ImuAccelZDelta301,ImuAccelZDelta301Exp2,ImuAccelZDelta301Exp3,Speed,Terrain,Trial
0,0.000621,3.859630e-07,2.397831e-10,0.010195,0.000104,1.059784e-06,0.020153,0.000406,0.000008,0.027217,0.000741,0.00002,0.037046,0.001372,0.000051,0.045782,0.002096,0.000096,0.054166,0.002934,0.000159,0.062247,0.003875,0.000241,0.074418,0.005538,0.000412,0.088505,0.007833,0.000693,0.095535,0.009127,0.000872,0.105079,0.011042,0.001160,0.112203,0.012590,0.001413,0.122862,0.015095,0.001855,0.130493,0.017028,0.002222,0.140431,0.019721,0.002769,0.150494,0.022648,0.003408,0.157233,0.024722,0.003887,0.168567,0.028415,0.004790,0.177887,0.031644,0.005629,...,-2.354098,5.541779,-13.045892,-0.283712,0.080493,-0.022837,-3.133411,9.818267,-30.764670,0.202311,0.040930,0.008280,-1.962646,3.851981,-7.560077,-1.888626,3.566907,-6.736553,-1.224035,1.498262,-1.833926,-1.383249,1.913379,-2.646680,-1.312022,1.721402,-2.258518,-2.908955,8.462017,-24.615623,-0.547673,0.299946,-0.164272,-0.708085,0.501384,-0.355023,-0.365115,0.133309,-0.048673,-2.274491,5.173311,-11.766650,0.575208,0.330864,0.190315,1.256956,1.579939,1.985914,-1.317110,1.734778,-2.284892,-1.206378,1.455348,-1.755700,-1.399111,1.957512,-2.738778,15,MitTile,8
1,0.000621,3.859630e-07,2.397831e-10,0.010110,0.000102,1.033444e-06,0.020078,0.000403,0.000008,0.027124,0.000736,0.00002,0.036957,0.001366,0.000050,0.045696,0.002088,0.000095,0.053751,0.002889,0.000155,0.062158,0.003864,0.000240,0.074421,0.005538,0.000412,0.088423,0.007819,0.000691,0.095453,0.009111,0.000870,0.104983,0.011022,0.001157,0.112114,0.012570,0.001409,0.122769,0.015072,0.001850,0.130277,0.016972,0.002211,0.140211,0.019659,0.002756,0.150282,0.022585,0.003394,0.157220,0.024718,0.003886,0.167951,0.028208,0.004738,0.177

### Speed filtering

In [17]:
df = df[df['Speed']==15]

### Logistic Regression
#### Now that we have processed the data, we can apply logistic regression to classify.

In [18]:
# sort the train test split where train on t1 test on t2
dfTrain = df[df['Trial']<9]
dfTest = df[df['Trial']>=9]

Y_train = dfTrain['Terrain']
Y_test = dfTest['Terrain']

X_train = dfTrain.drop(columns=['Terrain', 'Speed', 'Trial'])
X_test = dfTest.drop(columns=['Terrain', 'Speed', 'Trial'])

### Feature engineering: Kbest features

In [32]:
featureCount = 30
test = SelectKBest(k=featureCount)
fit = test.fit(X_train, Y_train)
# print(fit.scores_)
X_train = fit.transform(X_train)
X_test = fit.transform(X_test)

In [19]:
# randomly split to train and test
# # separate labels
# terrains = df['Terrain']
# speeds = df['Speed']
# df= df.drop(columns=['Terrain', 'Speed'])

# X = df
# Y = terrains
# X_test, X_train, Y_test, Y_train = train_test_split(X, Y, test_size=0.5)

In [20]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(123978, 1488)
(123978,)
(21375, 1488)
(21375,)


In [ ]:
model = LogisticRegression()
model.fit(X_train, Y_train)

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



In [ ]:
print("Accuracy on test set is: {}".format(model.score(X_test, Y_test)))